<a href="https://colab.research.google.com/github/lpdata/fraude_bilhetagem/blob/main/notebooks/03_modelagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelagem e Comparação de Modelos Interpretáveis para Detecção de Fraudes em Bilhetagem Eletrônica

# 1. SELEÇÃO E JUSTIFICATIVA DOS MODELOS CANDIDATOS

## 1.1 Descrição da Etapa

Antes da etapa de treinamento, é fundamental avaliar quais algoritmos de Machine Learning são mais adequados ao problema de detecção de fraude em sistemas de bilhetagem eletrônica, considerando as características do dataset e os requisitos do projeto.

Este problema consiste em uma **classificação binária**, com **classes desbalanceadas**, baseada em **dados estruturados/tabulares**, fortemente apoiados em **engenharia de features**. Além disso, há uma exigência explícita de **interpretabilidade**, uma vez que os resultados do modelo precisam ser compreensíveis e justificáveis em um contexto operacional.

Dessa forma, a escolha dos modelos não deve se basear apenas em performance preditiva, mas também em critérios como transparência, estabilidade e facilidade de comunicação dos resultados.

---

## 1.2 Comparação dos Modelos Candidatos

A seguir, são apresentados os principais modelos candidatos considerados para este problema, juntamente com uma análise qualitativa de seus principais prós e contras, bem como sua adequação ao contexto da detecção de fraudes.

| Modelo | Prós | Contras | Adequação |
|------|-----|---------|-----------|
| **Regressão Logística** | Alta interpretabilidade<br>Coeficientes explicáveis<br>Baseline robusto | Relações lineares<br>Depende de boas features | **Muito alta**<br>Baseline interpretável |
| **Árvore de Decisão** | Regras claras<br>Alta explicabilidade<br>Captura não linearidades | Sensível a ruído<br>Overfitting sem controle | **Alta**<br>Boa para explicação |
| **Random Forest** | Boa performance<br>Reduz overfitting<br>Interações complexas | Menor transparência<br>Custo computacional maior | **Alta**<br>Equilíbrio geral |
| Gradient Boosting | Forte poder preditivo<br>Bom em fraude | Complexidade elevada<br>Difícil explicação | Média |
| XGBoost / LightGBM | Performance de ponta<br>Robusto | Caixa-preta relativa<br>Difícil uso operacional | Média / Baixa |
| SVM | Bom em certos cenários | Pouco interpretável<br>Escala limitada | Baixa |
| kNN | Simples conceitualmente | Não escala bem<br>Difícil interpretação | Baixa |
| Naive Bayes | Rápido<br>Simples | Suposição forte<br>Baixa performance | Baixa |


---

## 1.3 Modelos Selecionados para a Etapa de Modelagem

Com base na análise comparativa apresentada, a seleção dos modelos para a etapa de modelagem considerou os seguintes critérios:

- Prioridade à **interpretabilidade**, conforme exigido pelo projeto  
- Adequação a **dados tabulares** e features engenheiradas  
- Capacidade de lidar com **desbalanceamento de classes**  
- Equilíbrio entre **capacidade preditiva** e **explicabilidade**  
- Facilidade de comunicação dos resultados em ambiente operacional  

Dessa forma, foram selecionados os seguintes modelos para a competição de performance:

### Modelos escolhidos

- **Regressão Logística**  
  Utilizada como baseline interpretável, permitindo análise direta do impacto das variáveis explicativas sobre a probabilidade de fraude.

- **Árvore de Decisão (com profundidade controlada)**  
  Empregada para capturar padrões não lineares de forma transparente, por meio de regras explícitas de decisão.

- **Random Forest**  
  Utilizada como modelo mais robusto, oferecendo ganho de performance ao mesmo tempo em que preserva mecanismos de explicabilidade baseados em importância das variáveis.

Essa combinação de modelos garante uma comparação justa entre abordagens lineares e não lineares, atendendo aos requisitos técnicos e metodológicos do projeto.


# 2. CONFIGURAÇÕES INICIAIS

## 2.1 Vinculação com Github

In [13]:
import os
from pathlib import Path

REPO_NAME = "fraude_bilhetagem"
REPO_URL = "https://github.com/lpdata/fraude_bilhetagem"

%cd /content

if not Path(REPO_NAME).exists():
    !git clone {REPO_URL}
else:
    print(f"Repo '{REPO_NAME}' já existe em /content. Pulando clone.")

%cd /content/{REPO_NAME}

!ls

!ls data/processed

print("Diretório atual:", os.getcwd())

/content
Repo 'fraude_bilhetagem' já existe em /content. Pulando clone.
/content/fraude_bilhetagem
data  notebooks  README.md  requirements.txt  src
dados_tratados.csv     metadados_dataset.json
metadados_colunas.csv  schema_dados_tratados.json
Diretório atual: /content/fraude_bilhetagem


In [14]:
# Definição de caminhos oficiais do notebook 03
DATA_PROCESSED_DIR = "data/processed"
DATASET_PATH = f"{DATA_PROCESSED_DIR}/dados_tratados.csv"
SCHEMA_PATH = f"{DATA_PROCESSED_DIR}/schema_dados_tratados.json"
METADADOS_COLUNAS_PATH = f"{DATA_PROCESSED_DIR}/metadados_colunas.csv"
METADADOS_DATASET_PATH = f"{DATA_PROCESSED_DIR}/metadados_dataset.json"

print("Dataset (modelagem):", DATASET_PATH)
print("Schema:", SCHEMA_PATH)
print("Metadados colunas:", METADADOS_COLUNAS_PATH)
print("Metadados dataset:", METADADOS_DATASET_PATH)

from pathlib import Path
paths = [DATASET_PATH, SCHEMA_PATH, METADADOS_COLUNAS_PATH, METADADOS_DATASET_PATH]
missing = [p for p in paths if not Path(p).exists()]
if missing:
    raise FileNotFoundError(f"Arquivos esperados não encontrados em {DATA_PROCESSED_DIR}: {missing}")

print("OK — artefatos de modelagem encontrados e prontos para uso.")

Dataset (modelagem): data/processed/dados_tratados.csv
Schema: data/processed/schema_dados_tratados.json
Metadados colunas: data/processed/metadados_colunas.csv
Metadados dataset: data/processed/metadados_dataset.json
OK — artefatos de modelagem encontrados e prontos para uso.


<small>***Comentários Letícia:** Neste ponto do projeto, já trabalho diretamente com o dataset processado, que concentra todas as decisões tomadas nas etapas anteriores de exploração, tratamento e engenharia de features. Optei por iniciar a modelagem a partir desse artefato final justamente para preservar o que já foi avaliado e validado, garantindo continuidade ao pipeline e evitando qualquer retorno desnecessário aos dados brutos. A verificação dos arquivos serve apenas para confirmar que estou partindo da base correta antes de avançar para a construção dos modelos.*</small>

## 2.2 Imports e configurações

In [15]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd

# Reprodutibilidade global
SEED = 42
np.random.seed(SEED)

# Configurações de exibição do notebook
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

# Imports para modelagem e pipelines
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Modelos selecionados
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Métricas de avaliação (fraude / desbalanceamento)
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

print("OK — imports carregados e configurações aplicadas.")
print("SEED:", SEED)

OK — imports carregados e configurações aplicadas.
SEED: 42


<small>***Comentários Letícia:** Nesta etapa, organizei os imports e defini as configurações iniciais do ambiente para garantir consistência ao longo de toda a modelagem. A definição da seed é importante para que os resultados sejam reproduzíveis e comparáveis entre os modelos, evitando variações causadas por aleatoriedade. Com isso, deixei o ambiente preparado para seguir com segurança para as próximas etapas de validação e treino dos modelos.*</small>

## 2.3 Carregamento dos dados e validação de integridade

### 2.3.1 Carregar dataset processado

In [16]:
df = pd.read_csv(DATASET_PATH)

print("Dataset carregado com sucesso.")
print("Shape do dataset:", df.shape)

df.head()

Dataset carregado com sucesso.
Shape do dataset: (30000, 42)


,id_transacao,id_cartao,ts_transacao,target_fraude,hora_transacao,dia_semana,data_transacao,fim_de_semana,tempo_vida_cartao_dias,tempo_desde_ultima_transacao_min,tempo_desde_ultima_transacao_horas,uso_intervalo_curto,qtd_transacoes_dia,qtd_transacoes_24h,uso_intenso_24h,linha_repetida,dispositivo_repetido,qtd_linhas_distintas_dia,qtd_dispositivos_distintos_dia,idade_suspeita,feriado_bin,feriado_nao_mapeado,temp_faixa,sentido_ida,clima_adverso,valor_transacao_faixa,cartao_qtd_transacoes,cartao_dias_ativos,cartao_media_transacoes_por_dia,cartao_qtd_linhas_distintas,cartao_qtd_dispositivos_distintos,cartao_qtd_motoristas_distintos,cartao_valor_transacao_mean,cartao_valor_transacao_std,cartao_pct_integracao,cartao_pct_feriado,cartao_pct_intervalo_curto,periodo_dia,valor_vs_media_cartao,valor_zscore_cartao,valor_outlier_cartao,uso_acima_media_dia_cartao
0,15192,10000,2026-01-24 10:54:15,1,10,5,2026-01-24,1,1848,NaN,NaN,0,1,1,0,0,0,1,1,0,0,0,25-30,0,1,baixo,2,2,1.0,2,2,2,4.5,0.000000,0.0,0.0,0.0,manha,0.0,0.000000,0,0
1,24059,10000,2026-01-29 10:03:32,0,10,3,2026-01-29,0,1089,7149.283333,119.154722,0,1,1,0,0,0,1,1,0,0,0,>30,0,1,baixo,2,2,1.0,2,2,2,4.5,0.000000,0.0,0.0,0.0,manha,0.0,0.000000,0,0
2,1599,10001,2026-01-03 04:46:30,0,4,5,2026-01-03,1,1388,NaN,NaN,0,1,1,0,0,0,1,1,0,0,0,25-30,1,0,baixo,5,5,1.0,5,5,5,3.6,2.012461,0.4,0.0,0.0,madrugada,3.6,-1.788854,0,0
3,17807,10001,2026-01-05 17:53:24,0,17,0,2026-01-05,0,920,3666.900000,61.115000,0,1,1,0,0,0,1,1,0,0,0,25-30,0,0,baixo,5,5,1.0,5,5,5,3.6,2.012461,0.4,0.0,0.0,tarde,0.9,0.447214,0,0
4,21208,10001,2026-01-16 02:22:42,0,2,4,2026-01-16,0,721,14909.300000,248.488333,0,1,1,0,0,0,1,1,0,0,0,25-30,1,1,baixo,5,5,1.0,5,5,5,3.6,2.012461,0.4,0.0,0.0,madrugada,0.9,0.447214,0,0


### 2.3.2 Validar colunas esperadas (schema check)

In [17]:
# lista esperada de colunas com base no artefato final do projeto
expected_columns = {
    "id_transacao",
    "id_cartao",
    "ts_transacao",
    "target_fraude",
    "hora_transacao",
    "dia_semana",
    "data_transacao",
    "fim_de_semana",
    "tempo_vida_cartao_dias",
    "tempo_desde_ultima_transacao_min",
    "tempo_desde_ultima_transacao_horas",
    "uso_intervalo_curto",
    "qtd_transacoes_dia",
    "qtd_transacoes_24h",
    "uso_intenso_24h",
    "linha_repetida",
    "dispositivo_repetido",
    "qtd_linhas_distintas_dia",
    "qtd_dispositivos_distintos_dia",
    "idade_suspeita",
    "feriado_bin",
    "feriado_nao_mapeado",
    "temp_faixa",
    "sentido_ida",
    "clima_adverso",
    "valor_transacao_faixa",
    "cartao_qtd_transacoes",
    "cartao_dias_ativos",
    "cartao_media_transacoes_por_dia",
    "cartao_qtd_linhas_distintas",
    "cartao_qtd_dispositivos_distintos",
    "cartao_qtd_motoristas_distintos",
    "cartao_valor_transacao_mean",
    "cartao_valor_transacao_std",
    "cartao_pct_integracao",
    "cartao_pct_feriado",
    "cartao_pct_intervalo_curto",
    "periodo_dia",
    "valor_vs_media_cartao",
    "valor_zscore_cartao",
    "valor_outlier_cartao",
    "uso_acima_media_dia_cartao"
}

current_columns = set(df.columns)

missing_columns = expected_columns - current_columns
extra_columns = current_columns - expected_columns

print("Colunas esperadas:", len(expected_columns))
print("Colunas no dataset:", len(current_columns))

if missing_columns:
    print("Colunas faltantes:", missing_columns)
else:
    print("Nenhuma coluna esperada está faltando.")

if extra_columns:
    print("Colunas extras encontradas:", extra_columns)
else:
    print("Nenhuma coluna extra encontrada.")

if missing_columns or extra_columns:
    raise ValueError("Schema de colunas divergente do esperado.")

print("OK — schema de colunas validado com sucesso.")

Colunas esperadas: 42
Colunas no dataset: 42
Nenhuma coluna esperada está faltando.
Nenhuma coluna extra encontrada.
OK — schema de colunas validado com sucesso.


### 2.3.3 Validar tipos e categorias (schema enforcement)

In [18]:
import json

# carregar schema oficial exportado
with open(SCHEMA_PATH, "r", encoding="utf-8") as f:
    schema = json.load(f)

expected_dtypes = schema["colunas"]

# mapa auxiliar: pandas dtype -> string padronizada
def normalize_dtype(dtype) -> str:
    return str(dtype)

# comparar tipos atuais vs esperados
type_mismatches = {}
for col, expected_type in expected_dtypes.items():
    if col not in df.columns:
        continue
    current_type = normalize_dtype(df[col].dtype)
    if current_type != expected_type:
        type_mismatches[col] = {"esperado": expected_type, "atual": current_type}

print("Total de colunas no schema:", len(expected_dtypes))
print("Total de colunas no dataframe:", df.shape[1])
print("Divergências de tipo encontradas:", len(type_mismatches))

if type_mismatches:
    display(pd.DataFrame(type_mismatches).T.sort_index())
else:
    print("OK — tipos compatíveis com o schema exportado.")

# checagem específica para categorias
expected_category_cols = [c for c, t in expected_dtypes.items() if t == "category"]
current_category_cols = [c for c in expected_category_cols if str(df[c].dtype) == "category"]

print("\nColunas esperadas como category:", expected_category_cols)
print("Colunas atualmente como category:", current_category_cols)

missing_category_cast = [c for c in expected_category_cols if str(df[c].dtype) != "category"]

if missing_category_cast:
    print("\nAtenção — colunas esperadas como category que não estão como category (pandas pode ter convertido ao ler CSV):")
    print(missing_category_cast)

    # aplicar cast defensivo
    for c in missing_category_cast:
        df[c] = df[c].astype("category")

    print("\nCast aplicado. Tipos atualizados para category nas colunas acima.")
else:
    print("\nOK — colunas categóricas já estão no tipo category.")

print("\nCheckpoint — dtypes atuais (amostra):")
display(df.dtypes.head(15))

Total de colunas no schema: 42
Total de colunas no dataframe: 42
Divergências de tipo encontradas: 14


,esperado,atual
cartao_media_transacoes_por_dia,Float64,float64
cartao_pct_integracao,Float64,float64
cartao_qtd_transacoes,Int64,int64
dia_semana,int32,int64
hora_transacao,int32,int64
id_cartao,Int64,int64
id_transacao,Int64,int64
periodo_dia,category,object
qtd_transacoes_24h,Int64,int64
qtd_transacoes_dia,Int64,int64



Colunas esperadas como category: ['temp_faixa', 'valor_transacao_faixa', 'periodo_dia']
Colunas atualmente como category: []

Atenção — colunas esperadas como category que não estão como category (pandas pode ter convertido ao ler CSV):
['temp_faixa', 'valor_transacao_faixa', 'periodo_dia']

Cast aplicado. Tipos atualizados para category nas colunas acima.

Checkpoint — dtypes atuais (amostra):


,0
id_transacao,int64
id_cartao,int64
ts_transacao,object
target_fraude,int64
hora_transacao,int64
dia_semana,int64
data_transacao,object
fim_de_semana,int64
tempo_vida_cartao_dias,int64
tempo_desde_ultima_transacao_min,float64


In [19]:
import pandas as pd

# aplicar casts com base no schema exportado
casts_aplicados = {}

for col, expected_type in expected_dtypes.items():
    if col not in df.columns:
        continue

    # datetime
    if expected_type == "datetime64[ns]":
        if df[col].dtype != "datetime64[ns]":
            df[col] = pd.to_datetime(df[col], errors="coerce")
            casts_aplicados[col] = {"para": expected_type}

    # category
    elif expected_type == "category":
        if str(df[col].dtype) != "category":
            df[col] = df[col].astype("category")
            casts_aplicados[col] = {"para": expected_type}

    # pandas nullable integer
    elif expected_type == "Int64":
        if str(df[col].dtype) != "Int64":
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
            casts_aplicados[col] = {"para": expected_type}

    # pandas nullable float
    elif expected_type == "Float64":
        if str(df[col].dtype) != "Float64":
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Float64")
            casts_aplicados[col] = {"para": expected_type}

    # numpy/int/float padrão
    else:

        pass

print("Casts aplicados:", len(casts_aplicados))
if casts_aplicados:
    display(pd.DataFrame(casts_aplicados).T.sort_index())

# revalidar divergências após enforcement
type_mismatches_after = {}
for col, expected_type in expected_dtypes.items():
    if col not in df.columns:
        continue
    current_type = str(df[col].dtype)
    if current_type != expected_type:
        type_mismatches_after[col] = {"esperado": expected_type, "atual": current_type}

print("\nDivergências restantes após enforcement:", len(type_mismatches_after))
if type_mismatches_after:
    display(pd.DataFrame(type_mismatches_after).T.sort_index())
else:
    print("OK — tipos alinhados ao schema exportado.")

Casts aplicados: 9


,para
cartao_media_transacoes_por_dia,Float64
cartao_pct_integracao,Float64
cartao_qtd_transacoes,Int64
id_cartao,Int64
id_transacao,Int64
qtd_transacoes_24h,Int64
qtd_transacoes_dia,Int64
target_fraude,Int64
ts_transacao,datetime64[ns]



Divergências restantes após enforcement: 2


,esperado,atual
dia_semana,int32,int64
hora_transacao,int32,int64


In [20]:
df["hora_transacao"] = df["hora_transacao"].astype("int32")
df["dia_semana"] = df["dia_semana"].astype("int32")

remaining = {
    col: {"esperado": expected_dtypes[col], "atual": str(df[col].dtype)}
    for col in ["hora_transacao", "dia_semana"]
    if str(df[col].dtype) != expected_dtypes[col]
}

print("Divergências restantes (hora_transacao/dia_semana):", len(remaining))
if remaining:
    display(pd.DataFrame(remaining).T)
else:
    print("OK — tipos 100% alinhados ao schema exportado.")

Divergências restantes (hora_transacao/dia_semana): 0
OK — tipos 100% alinhados ao schema exportado.


<small>***Comentários Letícia:** Nesta etapa, validei os tipos de dados do dataset com base no schema exportado nas fases anteriores e ajustei explicitamente as colunas que o Pandas não preserva ao ler arquivos CSV. Convertemos variáveis categóricas, datas e tipos numéricos para garantir aderência total ao schema definido, mantendo consistência com o pipeline já construído. Esse alinhamento é importante para evitar comportamentos inesperados na modelagem e reforça a integridade do conjunto de dados que será utilizado nos modelos.*</small>

### 2.3.4 Checagem de duplicidade e chaves

In [21]:
import pandas as pd

# checagem de duplicidade (linhas completas)
dup_rows = df.duplicated().sum()

# checagem por chaves lógicas
dup_id_transacao = df.duplicated(subset=["id_transacao"]).sum()
dup_id_cartao_ts = df.duplicated(subset=["id_cartao", "ts_transacao"]).sum()

print("Duplicatas (linhas completas):", dup_rows)
print("Duplicatas por id_transacao:", dup_id_transacao)
print("Duplicatas por (id_cartao, ts_transacao):", dup_id_cartao_ts)

# amostras (apenas se existir duplicidade)
if dup_rows > 0:
    print("\nAmostra de linhas duplicadas (linhas completas):")
    display(df[df.duplicated(keep=False)].head(10))

if dup_id_transacao > 0:
    print("\nAmostra de duplicatas por id_transacao:")
    display(df[df.duplicated(subset=["id_transacao"], keep=False)].sort_values("id_transacao").head(10))

if dup_id_cartao_ts > 0:
    print("\nAmostra de duplicatas por (id_cartao, ts_transacao):")
    display(df[df.duplicated(subset=["id_cartao", "ts_transacao"], keep=False)].sort_values(["id_cartao", "ts_transacao"]).head(10))

# checkpoint lógico
if dup_id_transacao > 0:
    raise ValueError("Encontradas duplicidades em id_transacao. Avaliar integridade da chave antes de prosseguir.")

print("\nOK — checagem de duplicidade concluída. Nenhuma duplicidade crítica em id_transacao.")

Duplicatas (linhas completas): 0
Duplicatas por id_transacao: 0
Duplicatas por (id_cartao, ts_transacao): 0

OK — checagem de duplicidade concluída. Nenhuma duplicidade crítica em id_transacao.


### 2.3.5 Sanity checks críticos para fraude

In [22]:
# distribuição do target
target_col = "target_fraude"

print("Distribuição do target (contagem):")
display(df[target_col].value_counts(dropna=False).to_frame("count"))

print("\nDistribuição do target (proporção):")
display((df[target_col].value_counts(normalize=True, dropna=False) * 100).round(4).to_frame("%"))

# nulos por coluna (top 15)
null_counts = df.isna().sum().sort_values(ascending=False)
null_pct = (null_counts / len(df) * 100).round(4)

sanity_nulls = pd.DataFrame({"nulos": null_counts, "%": null_pct})
print("\nNulos por coluna (top 15):")
display(sanity_nulls.head(15))

# checks de domínio lógico
issues = {}

# flags binárias esperadas
binary_cols = [
    "fim_de_semana", "uso_intervalo_curto", "uso_intenso_24h", "linha_repetida",
    "dispositivo_repetido", "idade_suspeita", "feriado_bin", "feriado_nao_mapeado",
    "sentido_ida", "clima_adverso", "valor_outlier_cartao", "uso_acima_media_dia_cartao"
]

# verificar se flags estão em {0,1} (ignorando nulos)
invalid_binary = {}
for c in binary_cols:
    if c in df.columns:
        vals = set(df[c].dropna().unique().tolist())
        if not vals.issubset({0, 1}):
            invalid_binary[c] = sorted(list(vals))

if invalid_binary:
    issues["flags_fora_0_1"] = invalid_binary

# checks de faixa
if "hora_transacao" in df.columns:
    invalid_hora = df.loc[(df["hora_transacao"] < 0) | (df["hora_transacao"] > 23), "hora_transacao"]
    if len(invalid_hora) > 0:
        issues["hora_transacao_fora_faixa"] = int(len(invalid_hora))

if "dia_semana" in df.columns:
    invalid_dia = df.loc[(df["dia_semana"] < 0) | (df["dia_semana"] > 6), "dia_semana"]
    if len(invalid_dia) > 0:
        issues["dia_semana_fora_faixa"] = int(len(invalid_dia))

# checks de tempo: não pode ser negativo quando não nulo
time_cols = ["tempo_desde_ultima_transacao_min", "tempo_desde_ultima_transacao_horas"]
invalid_time = {}
for c in time_cols:
    if c in df.columns:
        neg = df.loc[df[c].notna() & (df[c] < 0), c]
        if len(neg) > 0:
            invalid_time[c] = int(len(neg))
if invalid_time:
    issues["tempos_negativos"] = invalid_time

# checks simples de contagens: não negativas
count_cols = [
    "qtd_transacoes_dia", "qtd_transacoes_24h", "qtd_linhas_distintas_dia",
    "qtd_dispositivos_distintos_dia", "cartao_qtd_transacoes", "cartao_dias_ativos",
    "cartao_qtd_linhas_distintas", "cartao_qtd_dispositivos_distintos", "cartao_qtd_motoristas_distintos"
]
invalid_counts = {}
for c in count_cols:
    if c in df.columns:
        neg = df.loc[df[c].notna() & (df[c] < 0), c]
        if len(neg) > 0:
            invalid_counts[c] = int(len(neg))
if invalid_counts:
    issues["contagens_negativas"] = invalid_counts

# resumo de issues
print("\nSanity checks — resumo:")
if issues:
    display(pd.DataFrame({"issue": list(issues.keys()), "detalhe": list(issues.values())}))
    raise ValueError("Sanity checks falharam: há valores fora do domínio esperado. Revisar antes de prosseguir.")
else:
    print("OK — sanity checks concluídos (sem inconsistências de domínio detectadas).")

Distribuição do target (contagem):


,count
target_fraude,
0,27065
1,2935



Distribuição do target (proporção):


,%
target_fraude,
0,90.2167
1,9.7833



Nulos por coluna (top 15):


,nulos,%
tempo_desde_ultima_transacao_min,9483,31.61
tempo_desde_ultima_transacao_horas,9483,31.61
id_cartao,0,0.00
ts_transacao,0,0.00
hora_transacao,0,0.00
target_fraude,0,0.00
dia_semana,0,0.00
data_transacao,0,0.00
fim_de_semana,0,0.00
id_transacao,0,0.00



Sanity checks — resumo:
OK — sanity checks concluídos (sem inconsistências de domínio detectadas).


<small>***Comentários Letícia:** Nesta etapa, validei a consistência lógica do dataset já processado, verificando a distribuição da variável alvo, a presença de valores nulos e o domínio das principais variáveis utilizadas na modelagem. A proporção de fraudes confirma o desbalanceamento esperado do problema, enquanto os valores ausentes aparecem apenas em colunas onde esse comportamento já era previsto pelo contexto das features temporais. Os checks de domínio não indicaram inconsistências, o que reforça que o conjunto de dados está coerente e pronto para avançar para a definição formal de papéis das colunas e para a etapa de modelagem propriamente dita.*</small>


### 2.3.6 Definição formal de papéis das colunas

In [23]:
# colunas de rastreio (não entram na modelagem)
colunas_rastreio = [
    "id_transacao",
    "id_cartao",
    "ts_transacao",
    "data_transacao"
]

# coluna alvo
target_col = "target_fraude"

# features finais (todas as demais, excluindo rastreio e target)
features = [c for c in df.columns if c not in colunas_rastreio + [target_col]]

print("Resumo dos papéis das colunas:")
print("Colunas de rastreio:", len(colunas_rastreio))
print(colunas_rastreio)

print("\nColuna alvo:", target_col)

print("\nQuantidade de features:", len(features))
print(features)

# validações defensivas
assert target_col not in features, "Target não deve estar na lista de features."
assert all(c not in features for c in colunas_rastreio), "Colunas de rastreio não devem estar nas features."

# construção de X e y
X = df[features].copy()
y = df[target_col].copy()

print("\nShapes finais:")
print("X:", X.shape)
print("y:", y.shape)

print("\nOK — papéis das colunas definidos e X/y construídos com sucesso.")


Resumo dos papéis das colunas:
Colunas de rastreio: 4
['id_transacao', 'id_cartao', 'ts_transacao', 'data_transacao']

Coluna alvo: target_fraude

Quantidade de features: 37
['hora_transacao', 'dia_semana', 'fim_de_semana', 'tempo_vida_cartao_dias', 'tempo_desde_ultima_transacao_min', 'tempo_desde_ultima_transacao_horas', 'uso_intervalo_curto', 'qtd_transacoes_dia', 'qtd_transacoes_24h', 'uso_intenso_24h', 'linha_repetida', 'dispositivo_repetido', 'qtd_linhas_distintas_dia', 'qtd_dispositivos_distintos_dia', 'idade_suspeita', 'feriado_bin', 'feriado_nao_mapeado', 'temp_faixa', 'sentido_ida', 'clima_adverso', 'valor_transacao_faixa', 'cartao_qtd_transacoes', 'cartao_dias_ativos', 'cartao_media_transacoes_por_dia', 'cartao_qtd_linhas_distintas', 'cartao_qtd_dispositivos_distintos', 'cartao_qtd_motoristas_distintos', 'cartao_valor_transacao_mean', 'cartao_valor_transacao_std', 'cartao_pct_integracao', 'cartao_pct_feriado', 'cartao_pct_intervalo_curto', 'periodo_dia', 'valor_vs_media_carta

<small>***Comentários Letícia:** Nesta etapa, defini formalmente os papéis das colunas para a modelagem, separando explicitamente as variáveis de rastreio, a variável alvo e o conjunto final de features. As colunas de rastreio foram mantidas apenas para fins de auditoria e consistência, sem participar do treino dos modelos, enquanto o conjunto de features reflete exatamente o resultado da engenharia realizada nas etapas anteriores. Com a construção de `X` e `y`, deixei a base preparada para a etapa de divisão treino/teste e para a comparação justa entre os modelos.*</small>

### 2.3.7 Split treino/teste estratificado

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=SEED
)

print("Shapes do split:")
print("X_train:", X_train.shape, "| y_train:", y_train.shape)
print("X_test :", X_test.shape,  "| y_test :", y_test.shape)

train_rate = (y_train.mean() * 100)
test_rate = (y_test.mean() * 100)

print("\nProporção de fraude (%):")
print(f"Treino: {train_rate:.4f}%")
print(f"Teste : {test_rate:.4f}%")

print("\nOK — split estratificado concluído com sucesso.")

Shapes do split:
X_train: (24000, 37) | y_train: (24000,)
X_test : (6000, 37) | y_test : (6000,)

Proporção de fraude (%):
Treino: 9.7833%
Teste : 9.7833%

OK — split estratificado concluído com sucesso.


<small>***Comentários Letícia:** Realizei a divisão dos dados em treino e teste utilizando split estratificado para preservar a proporção da classe de fraude em ambos os conjuntos. Essa decisão é importante em um problema desbalanceado como este, pois garante que a avaliação dos modelos no conjunto de teste seja representativa do cenário real. Com os conjuntos separados e validados, a base está pronta para iniciar a etapa de modelagem sem risco de viés na comparação de desempenho.*</small>

### 2.3.8 Checkpoint final da etapa

In [25]:
checks = {
    "dataset_shape": df.shape == (30000, 42),
    "X_shape": X.shape == (30000, 37),
    "y_shape": y.shape == (30000,),
    "split_shapes": (
        X_train.shape == (24000, 37)
        and X_test.shape == (6000, 37)
        and y_train.shape == (24000,)
        and y_test.shape == (6000,)
    ),
    "target_balance_train_test": y_train.mean() == y_test.mean(),
    "sem_nulos_no_target": y.isna().sum() == 0,
}

print("Checkpoint final — validações:")
for k, v in checks.items():
    print(f"{k}: {'OK' if v else 'ERRO'}")

if not all(checks.values()):
    raise ValueError("Checkpoint final da etapa 2.3 falhou. Revisar validações antes de prosseguir.")

print("\nOK — etapa 2.3 concluída. Dataset íntegro e pronto para a modelagem.")


Checkpoint final — validações:
dataset_shape: OK
X_shape: OK
y_shape: OK
split_shapes: OK
target_balance_train_test: OK
sem_nulos_no_target: OK

OK — etapa 2.3 concluída. Dataset íntegro e pronto para a modelagem.


- Nesta etapa, foi realizada a validação completa do dataset que será utilizado na fase de modelagem. O conjunto de dados carregado corresponde ao artefato final do pipeline de tratamento e engenharia de features, previamente auditado e versionado.

- Foram verificadas a integridade estrutural do dataset, a aderência ao schema definido, os tipos de dados, a consistência lógica das variáveis e a ausência de duplicidades em chaves críticas. Também foram realizados sanity checks específicos para o contexto de fraude, confirmando a distribuição esperada da variável alvo e a inexistência de valores fora do domínio lógico.

- Em seguida, os papéis das colunas foram definidos de forma explícita, separando variáveis de rastreio, variável alvo e conjunto final de features. O dataset foi então dividido em conjuntos de treino e teste por meio de split estratificado, preservando a proporção da classe de fraude em ambos os conjuntos.

- Com todas as validações concluídas com sucesso, o dataset encontra-se íntegro, consistente e pronto para a etapa de modelagem, garantindo que a comparação entre os modelos seja realizada sobre uma base confiável e representativa.


# 3. MODELO 1 - REGRESSÃO LOGÍSTICA

## 3.1 Objetivo do Modelo

## 3.2 Construção do pipeline

### 3.2.1 Pré-processamento

### 3.2.2 Padronização (scaler) onde faz sentido

### 3.2.3 Tratamento de desbalanceamento

### 3.2.4 Definição do estimador e hiperparâmetros iniciais

## 3.3 Avaliação (validação cruzada)

### 3.3.1 Definir estratégia de CV

### 3.3.2 Definir métricas oficiais do projeto

### 3.3.3 Executar CV e registrar resultados

## 3.4 Treino final e avaliação no holdout

### 3.4.1 Ajustar pipeline no treino completo

### 3.4.2 Avaliar no teste com métricas e matriz de confusão

## 3.5 Interpretabilidade

### 3.5.1 Extrair nomes pós-encoding e coeficientes

### 3.5.2 Ranking e leitura de sinais

## 3.6 Insights Finais do Modelo


# 4. MODELO 2 - ÁRVORE DE DECISÃO

## 4.1 Objetivo do Modelo

## 4.2 Construção do pipeline

### 4.2.1 Pré-processamento

### 4.2.2 Regularização explícita (anti-overfitting)

## 4.3 Avaliação (CV)

## 4.4 Treino final e holdout

## 4.5 Interpretabilidade

## 4.6 Insights Finais do Modelo

# 5. MODELO 3 - RANDOM FOREST

## 5.1 Objetivo do Modelo

## 5.2 Construção do pipeline

### 5.2.1 Pré-processamento padronizado

### 5.2.2 Definição de hiperparâmetros base

## 5.3 Avaliação (CV)

## 5.4 Treino final e holdout

## 5.5 Interpretabilidade

## 5.6 Insights Finais do Modelo

# 6. COMPETIÇÃO ENTRE MODELOS

## 6.1 Objetivos

## 6.2 Consolidar resultados de CV

## 6.3 Consolidar resultados no teste (holdout)

## 6.4 Análise de trade-off operacional

## 6.5 Checkpoint - Modelos Vencedor

# 7. RESULTADOS FINAIS